In [ ]:
# Create a S3 Bucket
# From the aws console or Boto3

# Create a bucket

In [10]:
import boto3
def create_s3_bucket(bucket_name, region='us-east-1'):
    try:
        s3 = boto3.resource('s3', region_name=region)
        
        # For us-east-1, no LocationConstraint should be specified
        if region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': region
                }
            )
        print(f"S3 bucket '{bucket_name}' has been created in region '{region}'")
        return True
        
    except Exception as e:
        print(f"Error creating bucket: {str(e)}")
        return False

# Example usage
bucket_name = "manuelfinancebucket"
region = "us-east-2"  # Change this to your desired region
create_s3_bucket(bucket_name, region)

S3 bucket 'manuelfinancebucket' has been created in region 'us-east-2'


True

In [12]:
!pip install yfinance
import pandas as pd
from datetime import datetime
import yfinance as yf

#parameters
start = datetime(2019, 1, 1)
end = datetime(2021, 1, 1)

#get data
df = yf.download('AAPL', start = start, end = end)
df.reset_index(inplace=True)
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 26.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 71.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp310-cp310-linux_x86_64.whl size=302883 sha256=b172e221e3e1c7e56be370117f718207f4cc5f68c23b8e229f032a174ee309b8
  Stored in directory: /home/ec2-user/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee


[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
0,2019-01-02 00:00:00+00:00,37.708591,39.480000,39.712502,38.557499,38.722500,148158800
1,2019-01-03 00:00:00+00:00,33.952541,35.547501,36.430000,35.500000,35.994999,365248800
2,2019-01-04 00:00:00+00:00,35.401947,37.064999,37.137501,35.950001,36.132500,234428400
3,2019-01-07 00:00:00+00:00,35.323158,36.982498,37.207500,36.474998,37.174999,219111200
4,2019-01-08 00:00:00+00:00,35.996529,37.687500,37.955002,37.130001,37.389999,164101200
...,...,...,...,...,...,...,...
500,2020-12-24 00:00:00+00:00,129.047546,131.970001,133.460007,131.100006,131.320007,54930100
501,2020-12-28 00:00:00+00:00,133.663010,136.690002,137.339996,133.509995,133.990005,124486200
502,2020-12-29 00:00:00+00:00,131.883316,134.869995,138.789993,134.339996,138.050003,121047300


# ETL

In [14]:
df.drop(columns=['Adj Close', 'Date'], inplace=True)

KeyError: 'Adj Close'

In [16]:
df_data_features = df.iloc[:-1 , :]
df_data_features

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
0,39.480000,39.712502,38.557499,38.722500,148158800
1,35.547501,36.430000,35.500000,35.994999,365248800
2,37.064999,37.137501,35.950001,36.132500,234428400
3,36.982498,37.207500,36.474998,37.174999,219111200
4,37.687500,37.955002,37.130001,37.389999,164101200
...,...,...,...,...,...
499,130.960007,132.429993,130.779999,132.160004,88223700
500,131.970001,133.460007,131.100006,131.320007,54930100
501,136.690002,137.339996,133.509995,133.990005,124486200


In [19]:
df_data_targets = df.iloc[1: , 0].rename("Targets")
3df_data_targets

1       35.547501
2       37.064999
3       36.982498
4       37.687500
5       38.327499
          ...    
500    131.970001
501    136.690002
502    134.869995
503    133.720001
504    132.690002
Name: Targets, Length: 504, dtype: float64

In [20]:
# For XGBoost to work the target column must be in the first column
df_data_features['Target'] = list(df_data_targets)
first_column = df_data_features.pop('Target')
df_data_features.insert(0,'Target', first_column)

df_data_final = df_data_features
df_data_final

/tmp/ipykernel_8593/4141939672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_targets)


Price,Target,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
1,35.547501,35.547501,36.430000,35.500000,35.994999,365248800
2,37.064999,37.064999,37.137501,35.950001,36.132500,234428400
3,36.982498,36.982498,37.207500,36.474998,37.174999,219111200
4,37.687500,37.687500,37.955002,37.130001,37.389999,164101200
5,38.327499,38.327499,38.632500,37.407501,37.822498,180396400
...,...,...,...,...,...,...
500,131.970001,131.970001,133.460007,131.100006,131.320007,54930100
501,136.690002,136.690002,137.339996,133.509995,133.990005,124486200
502,134.869995,134.869995,138.789993,134.339996,138.050003,121047300


# 

In [21]:
import numpy as np
df_randomized = df_data_final.sample(frac=1, random_state=123)
df_randomized

Price,Target,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
430,112.129997,112.129997,116.000000,112.040001,115.230003,154679000
283,79.750000,79.750000,79.937500,78.652496,78.839996,152531200
384,95.477501,95.477501,99.955002,95.257500,97.264999,191649200
478,113.849998,113.849998,117.620003,113.750000,117.180000,127959300
429,115.540001,115.540001,118.830002,113.610001,118.330002,184642000
...,...,...,...,...,...,...
99,44.742500,44.742500,45.535000,44.654999,45.049999,94858800
477,117.339996,117.339996,118.769997,117.290001,118.639999,73604300
323,71.107498,71.107498,71.582497,70.157501,70.599998,131154400


In [22]:
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(403, 6) (101, 6)


# Set a path and upload dataset into a S3 Bucket

In [24]:
import os
prefix = ("xgboost algo on the go")
train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, "train", "train.csv")
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, "test", "test.csv")
print(train_csv_path)
print(test_csv_path)

s3://manuelfinancebucket/xgboost algo on the go/train/train.csv
s3://manuelfinancebucket/xgboost algo on the go/test/test.csv


In [25]:
train_data.to_csv(train_csv_path, index = False, header = False)
test_data.to_csv(test_csv_path, index = False, header = False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


# Build an XGBoost Model
## You can use XGBoost as a Framework or as an Algorithm

In [27]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Find XGBoost image_uri and builds an XGBoost Container

In [29]:
xgboost_container = image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-2")
display(xgboost_container)

'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.2-2'

# Hyperparameters
| **Parameter Name**         | **Description**                                                                                                                                                  | **Required/Optional** | **Valid Values**                            | **Default Value**               |
|-----------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------|---------------------------------------------|----------------------------------|
| `num_class`                | The number of classes. Required if `objective` is set to `multi:softmax` or `multi:softprob`.                                                                   | Required               | Integer                                     | -                                |
| `num_round`                | The number of rounds to run the training.                                                                                                                       | Required               | Integer                                     | -                                |
| `alpha`                    | L1 regularization term on weights. Increasing this value makes models more conservative.                                                                        | Optional               | Float                                       | 0                                |
| `base_score`               | The initial prediction score of all instances, global bias.                                                                                                     | Optional               | Float                                       | 0.5                              |
| `booster`                  | Which booster to use: `gbtree`, `gblinear`, or `dart`.                                                                                                          | Optional               | `"gbtree"`, `"gblinear"`, `"dart"`          | `"gbtree"`                      |
| `colsample_bylevel`        | Subsample ratio of columns for each split, in each level.                                                                                                       | Optional               | Float, [0,1]                                | 1                                |
| `colsample_bynode`         | Subsample ratio of columns from each node.                                                                                                                      | Optional               | Float, (0,1]                                | 1                                |
| `colsample_bytree`         | Subsample ratio of columns when constructing each tree.                                                                                                         | Optional               | Float, [0,1]                                | 1                                |
| `csv_weights`              | Differentiates importance of instances for CSV input by taking the second column as instance weights.                                                          | Optional               | 0 or 1                                      | 0                                |
| `deterministic_histogram`  | Builds histogram on GPU deterministically. Used only if `tree_method` is set to `gpu_hist`.                                                                     | Optional               | `"true"`, `"false"`                         | `"true"`                        |
| `early_stopping_rounds`    | Stops training if validation score doesn't improve after specified rounds.                                                                                      | Optional               | Integer                                     | -                                |
| `eta`                      | Step size shrinkage to prevent overfitting.                                                                                                                     | Optional               | Float, [0,1]                                | 0.3                              |
| `eval_metric`              | Evaluation metric for validation data. Defaults depend on the `objective`.                                                                                      | Optional               | String                                      | Based on `objective`             |
| `gamma`                    | Minimum loss reduction required for further partitioning.                                                                                                       | Optional               | Float, [0,∞)                                | 0                                |
| `grow_policy`              | Controls how new nodes are added to the tree (`depthwise` or `lossguide`).                                                                                      | Optional               | `"depthwise"`, `"lossguide"`               | `"depthwise"`                   |
| `interaction_constraints`  | Groups of variables allowed to interact.                                                                                                                        | Optional               | Nested list of integers                    | None                             |
| `lambda`                   | L2 regularization term on weights.                                                                                                                              | Optional               | Float                                       | 1                                |
| `lambda_bias`              | L2 regularization term on bias.                                                                                                                                 | Optional               | Float, [0.0, 1.0]                          | 0                                |
| `max_bin`                  | Maximum number of bins for continuous features. Used with `tree_method=hist`.                                                                                   | Optional               | Integer                                     | 256                              |
| `max_delta_step`           | Maximum delta step for weight estimation. Helps control updates in logistic regression.                                                                         | Optional               | Integer, [0,∞)                              | 0                                |
| `max_depth`                | Maximum depth of trees. Increasing can cause overfitting.                                                                                                       | Optional               | Integer, [0,∞)                              | 6                                |
| `max_leaves`               | Maximum number of nodes added when `grow_policy=lossguide`.                                                                                                     | Optional               | Integer                                     | 0                                |
| `min_child_weight`         | Minimum sum of instance weight (hessian) needed in a child.                                                                                                     | Optional               | Float, [0,∞)                                | 1                                |
| `monotone_constraints`     | Specifies monotonicity constraints on features.                                                                                                                 | Optional               | Tuple of integers (-1, 0, 1)               | (0, 0)                          |
| `normalize_type`           | Type of normalization algorithm (`tree` or `forest`).                                                                                                          | Optional               | `"tree"`, `"forest"`                       | `"tree"`                        |
| `nthread`                  | Number of parallel threads used by XGBoost.                                                                                                                     | Optional               | Integer                                     | Maximum threads                  |
| `objective`                | Specifies the learning task and objective. Examples: `reg:squarederror`, `multi:softmax`.                                                                       | Optional               | String                                      | `"reg:squarederror"`             |
| `one_drop`                 | Enables dropping at least one tree during dropout.                                                                                                             | Optional               | 0 or 1                                      | 0                                |
| `process_type`             | Type of boosting process (`default` or `update`).                                                                                                              | Optional               | `"default"`, `"update"`                    | `"default"`                     |
| `rate_drop`                | Dropout rate specifying fraction of trees to drop.                                                                                                              | Optional               | Float, [0.0, 1.0]                          | 0.0                              |
| `refresh_leaf`             | Updates tree leaves and stats when enabled.                                                                                                                     | Optional               | 0 or 1                                      | 1                                |
| `sample_type`              | Type of sampling algorithm (`uniform` or `weighted`).                                                                                                           | Optional               | `"uniform"`, `"weighted"`                  | `"uniform"`                     |
| `scale_pos_weight`         | Balances positive and negative weights for unbalanced classes.                                                                                                  | Optional               | Float                                       | 1                                |
| `seed`                     | Random number seed.                                                                                                                                             | Optional               | Integer                                     | 0                                |
| `single_precision_histogram` | Uses single precision for histograms with `tree_method=hist` or `gpu_hist`.                                                                                    | Optional               | `"true"`, `"false"`                         | `"false"`                       |
| `sketch_eps`               | Approximation for greedy algorithm.                                                                                                                             | Optional               | Float, [0, 1]                               | 0.03                             |
| `skip_drop`                | Probability of skipping dropout during boosting.                                                                                                               | Optional               | Float, [0.0, 1.0]                          | 0.0                              |
| `subsample`                | Subsample ratio of training instances.                                                                                                                          | Optional               | Float, [0,1]                                | 1                                |
| `tree_method`              | Tree construction algorithm (`auto`, `exact`, `approx`, `hist`, `gpu_hist`).                                                                                    | Optional               | String                                      | `"auto"`                        |
| `tweedie_variance_power`   | Controls variance of Tweedie distribution.                                                                                                                      | Optional               | Float, (1, 2)                               | 1.5                              |
| `updater`                  | Comma-separated string defining tree updaters.                                                                                                                 | Optional               | String                                      | `"grow_colmaker, prune"`         |
| `use_dask_gpu_training`    | Enables Dask-based distributed GPU training (for versions ≥ 1.5-1).                                                                                             | Optional               | `"true"`, `"false"`                         | `"false"`                       |
| `verbosity`                | Verbosity level of printed messages (0: silent, 1: warning, 2: info, 3: debug).                                                                                 | Optional               | Integer                                     | 1                                |


In [35]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "early_stopping_rounds":10,
        "num_round":1000,
        }

# Set an output path where the model will be saved

In [32]:
                                       #bucket name / prefix / output folder    
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix, "output")
print(output_path)

s3://manuelfinancebucket/xgboost algo on the go/output/


# Construct an **Sagemaker Estimator** that calls **xgboost container**

In [36]:
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m4.xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                         use_spot_instances = True,
                                         max_run = 300,
                                         max_wait = 600
                                         )

# Define the data type and paths for the trainining and validation datasets

In [42]:
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)

# Execute XGBoost Training

In [43]:
estimator.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-11-19-17-49-50-146


2024-11-19 17:49:53 Starting - Starting the training job......
2024-11-19 17:50:27 Starting - Preparing the instances for training...
2024-11-19 17:51:03 Downloading - Downloading input data...
2024-11-19 17:51:29 Downloading - Downloading the training image......
2024-11-19 17:52:45 Training - Training image download completed. Training in progress..[2024-11-19 17:52:56.840 ip-10-0-246-114.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-11-19:17:52:56:INFO] Imported framework sagemaker_xgboost_container.training
[2024-11-19:17:52:56:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-11-19:17:52:56:INFO] No GPUs detected (normal if no gpus installed)
[2024-11-19:17:52:56:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-11-19:17:52:56:INFO] Determined delimiter of CSV input is ','
[2024-11-19:17:52:56:INFO] Determined delimiter of CSV input is ','
[2024-11-19:17:52:56:INFO] Deter

# Deploy Trained XGBoost Model as Endpoint
- Environment
    - Inside Sagemaker: Serialization by User
    - **Outside Sagemaker: Serialization by Endpoint**
| **Scenario**         | **Who Handles Serialization?**                              | **Example Format** |
|-----------------------|------------------------------------------------------------|--------------------|
| **Inside SageMaker**  | User must serialize data into a format like LibSVM, JSON, or CSV. | LibSVM            |
| **Outside SageMaker** | Endpoint automatically serializes input data.              | JSON, CSV          |

- Method to invoke endpoint
    - **API (single prediction)**
    - S3 (Batch prediction)
    
- Data type based on method
    - **API (JSON)**
    - s3 (CSV)
  

### To host a model in EC2 call deploy method of the xgb_model estimator
initial_instance_count(int) - The number of instances to deploy the model
instance_type(str) - The type of instances that you want to operate your deployed model
serializer(int) - Serialize input data of various formats (a Numpy array, list, file, or buffer) to a csv formatted string. We use this because the XGBosost algorithm acceps input files in CSV format


In [45]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge',serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-11-19-22-02-02-602
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-11-19-22-02-02-602
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-11-19-22-02-02-602


------!

In [46]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-11-19-22-02-02-602'

# Make a prediction using endpoints

In [49]:
# start with parameters
start_date = datetime(2021,1,4)
end_date = datetime(2021,1,5)

#get the data
df = yf.download('AAPL', start = start_date, end = end_date)
df.reset_index(inplace=True)
df

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
0,2021-01-04 00:00:00+00:00,126.544243,129.410004,133.610001,126.760002,133.520004,143301900


In [51]:
df.drop(axis=1, columns=['Adj Close'], inplace=True)
df.drop(axis=1, columns=['Date'], inplace=True)
data_features_array = df.values
data_features_array

/tmp/ipykernel_8593/599336121.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(axis=1, columns=['Adj Close'], inplace=True)
/tmp/ipykernel_8593/599336121.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(axis=1, columns=['Date'], inplace=True)


array([[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08]])

# Serialize data

In [52]:
Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
print(Y_pred_Fcn, type(Y_pred_Fcn))

131.77243041992188
 <class 'str'>


# Lambda function handler

In [58]:
import boto3
ENDPOINT_NAME = 'sagemaker-xgboost-2024-11-19-22-02-02-602'
runtime = boto3.client('runtime.sagemaker')
def lambda_handler(event, context):
    inputs = event['data']
    result = []
    for input in inputs:
        serialized_input = ','.join(map(str, input))
        response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                                      ContentType='text/csv',
                                      Body=serialized_input)
        result.append(response['Body'].read().decode())
    
    return result

['131.77243041992188\n', '131.77243041992188\n', '131.77243041992188\n']

In [ ]:

Input_json = { 'data':
        [[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08],[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08],[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08]] 
}
result = lambda_handler(Input_json, __)
result

# Requests API


In [59]:
import requests

API_ENDPOINT = "https://j5nhs5qh13.execute-api.us-east-2.amazonaws.com/xgboostmodel"

#data to be sent
json = {"data":
[[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08],[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08],[1.29410004e+02, 1.33610001e+02, 1.26760002e+02, 1.33520004e+02,
        1.43301900e+08]]
}

r = requests.post(url = API_ENDPOINT, json = json)

## with the lambda integration, the lambda function output is returned as a 200 OK response

In [60]:
print(f"Status Code: {r.status_code}, Response: {r.json()}")

Status Code: 200, Response: ['131.77243041992188\n', '131.77243041992188\n', '131.77243041992188\n']
